# Agents

In [ ]:
pip install ollama

In [ ]:
#!ollama serve

## Test simple

In [10]:
from litellm import completion

response = completion(
    model="ollama/deepseek-r1:32b", 
    messages=[{ "content": "respond in 20 words. who are you?","role": "user"}], 
    api_base="http://localhost:11434"
)
print(response.choices[0].message.content)


I'm DeepSeek-R1-Lite-Preview, an AI assistant created by DeepSeek. I provide helpful, honest answers across various domains while maintaining a formal tone.


## Test agents

In [12]:
from crewai import Agent, Task, Crew
from langchain_community.llms import Ollama

# Inicializamos el modelo local
ollama_model = Ollama(model="ollama/deepseek-r1:32b", base_url="http://localhost:11434")

# Creamos un agente con CrewAI
agent = Agent(
    role='Analyst',
    goal='Analizar el texto y detectar sesgos de género',
    backstory="Eres un experto en igualdad y lenguaje inclusivo.",
    llm=ollama_model
)

# Creamos una tarea
task = Task(
    description="Analiza el siguiente texto: 'Las mujeres son malas conductoras.'",
    expected_output="Un análisis del sesgo de género en el texto.",
    agent=agent
)

# Ejecutamos el crew (puede ser un solo agente o varios)
crew = Crew(agents=[agent], tasks=[task])
result = crew.kickoff()
print(result)

El análisis del sesgo de género en el texto "Las mujeres son malas conductoras" se puede realizar de la siguiente manera:

1. **Uso de términos generalizadores**: La frase utiliza "las mujeres", que refiere a todas las mujeres sin considerar individualidades, lo que puede contribuir a una percepción estereotipada.

2. **Judicialidad negativa**: El término "malas" se emplea para calificar las habilidades de conducción, lo que puede reforzar la idea de que las mujeres son incompetentes en este ámbito.

3. **Especificación del género en el rol**: La palabra "conductoras" relaciona directamente el género femenino con una supuesta incapacidad, refiriendo a mujeres como si fueran inherentemente peores conductoras que los hombres.

Este tipo de lenguaje puede perpetuar estereotipos y sesgos de género al vincular negativamente la condición femenina con una actividad considerada tradicionalmente masculina.


In [15]:
from langchain_community.chat_models import ChatOllama
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

# === 1. Modelo local ===
ollama_model = Ollama(model="deepseek-r1:32b", base_url="http://localhost:11434")

# === 2. Variables ===
VARIABLES = [
    'cita_textual_titular', 'genero_nombre_propio_titular', 'genero_periodista',
    'genero_personas_mencionadas', 'nombre_propio_titular', 'personas_mencionadas', 'tema'
]

CITA_TITULAR = {'0': 'No', '1': 'Sí'}
GENERO_NOMBRE_PROPIO_TITULAR = {'1': 'No hay', '2': 'Sí, hombre', '3': 'Sí, mujer', '4': 'Sí, mujer y hombre'}
GENERO_PERIODISTA = {'1': 'Masculino', '2': 'Femenino', '3': 'Mixto', '4': 'Ns/Nc', '5': 'Agencia/otros medios', '6': 'Redacción', '7': 'Corporativo'}
GENERO_PERSONAS_MENCIONADAS = {'1': 'No hay', '2': 'Sí, hombre', '3': 'Sí, mujer', '4': 'Sí, mujer y hombre'}
TEMA = {'1': 'Científica/Investigación', '2': 'Comunicación', '3': 'De farándula o espectáculo', '4': 'Deportiva', '5': 'Economía', '6': 'Educación/cultura', '7': 'Empleo/Trabajo', '8': 'Empresa', '9': 'Judicial', '10': 'Medioambiente', '11': 'Policial', '12': 'Política', '13': 'Salud', '14': 'Social', '15': 'Tecnología', '16': 'Transporte', '17': 'Otros'}

# === 3. Esquema ===
schemas = [
    ResponseSchema(
        name=var,
        description="Objeto con 'codigo' (string) y 'evidencia' (lista de fragmentos textuales que justifican el valor)."
    )
    for var in VARIABLES
]

output_parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = output_parser.get_format_instructions()

# === 4. Prompt ===
template_parts = [
    "Analiza el siguiente texto periodístico y clasifícalo según las variables de CONTENIDO_GENERAL.",
    "",
    "Texto:",
    "{texto}",
    "",
    "Para cada variable, devuelve un objeto con:",
    '- "codigo": el número correspondiente según las tablas (no inventes etiquetas ni palabras).',
    '- "evidencia": lista de fragmentos textuales del texto que justifican el valor.',
    "",
    "Usa *exclusivamente* los siguientes valores numéricos:",
    f"CITA_TITULAR = {list(CITA_TITULAR.keys())}",
    f"GENERO_NOMBRE_PROPIO_TITULAR = {list(GENERO_NOMBRE_PROPIO_TITULAR.keys())}",
    f"GENERO_PERIODISTA = {list(GENERO_PERIODISTA.keys())}",
    f"GENERO_PERSONAS_MENCIONADAS = {list(GENERO_PERSONAS_MENCIONADAS.keys())}",
    f"TEMA = {list(TEMA.keys())}",
    "",
    "Ejemplo de formato esperado:",
    "{{",
    '  "cita_textual_titular": {{"codigo": "1", "evidencia": ["\'María López presentó un estudio\'"]}},',
    '  "genero_personas_mencionadas": {{"codigo": "4", "evidencia": ["María López", "Juan Pérez"]}}',
    "}}",
    "",
    "{format_instructions}"
]

template = "\n".join(template_parts)

prompt = PromptTemplate(
    template=template,
    input_variables=["texto"],
    partial_variables={"format_instructions": format_instructions},
)

# === 5. Texto de prueba ===
text = """La científica María López presentó un nuevo estudio sobre inteligencia artificial,
pero el titular mencionó solo a su colega Juan Pérez."""

_input = prompt.format_prompt(texto=text)
output = ollama_model.invoke(_input.to_string())
structured = output_parser.parse(output.content)
print(structured)


AttributeError: 'str' object has no attribute 'content'

## Tools

In [6]:
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType
import requests

# === Tool: genderize.io ===
@tool("genderize_name")
def genderize_name(name: str) -> str:
    """Dado un nombre de pila, devuelve el género probable usando genderize.io."""
    try:
        r = requests.get("https://api.genderize.io", params={"name": name}, timeout=10)
        r.raise_for_status()
        data = r.json()
        # Resumimos para mostrar información útil
        return str({
            "name": data.get("name"),
            "gender": data.get("gender"),
            "probability": data.get("probability"),
            "count": data.get("count"),
        })
    except Exception as e:
        return f"Error consultando genderize: {e}"

# === Agente con herramientas (ReAct) ===
tools = [genderize_name]
react_agent = initialize_agent(
    tools=tools,
    llm=ollama_model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Ejemplo de uso
query = "Usa la herramienta para predecir el género de 'María' y 'Juan'."
react_result = react_agent.invoke({"input": query})
print(react_result["output"])


/tmp/ipykernel_822382/3128448210.py:25: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  react_agent = initialize_agent(




> Entering new AgentExecutor chain...
I need to use the `genderize_name` tool to determine the probable gender for each name.
Action: genderize_name
Action Input: María
Observation: {'name': 'María', 'gender': 'female', 'probability': 0.99, 'count': 97652}
Thought:Now I know the gender of María.
Action: genderize_name
Action Input: Juan
Observation: {'name': 'Juan', 'gender': 'male', 'probability': 0.99, 'count': 2143485}
Thought:Now I know the gender of Juan.
Final Answer: María es femenino y Juan es masculino.

> Finished chain.
María es femenino y Juan es masculino.


In [8]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType

# === Tool: Wikipedia (ES) ===
wiki = WikipediaAPIWrapper(lang="es", top_k_results=3, doc_content_chars_max=2000)

@tool("wikipedia_search")
def wikipedia_search(query: str) -> str:
    """Busca en Wikipedia (es) y devuelve un resumen de los mejores resultados."""
    try:
        result = wiki.run(query)
        return result if isinstance(result, str) else str(result)
    except Exception as e:
        return f"Error en Wikipedia: {e}"

# === Agente con herramientas (incluye Wikipedia) ===
tools = [genderize_name, wikipedia_search]
react_agent = initialize_agent(
    tools=tools,
    llm=ollama_model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Ejemplo de uso
query = "Busca en Wikipedia quién es 'Marie Curie' y resume en 2 frases."
react_result = react_agent.invoke({"input": query})
print(react_result["output"])




> Entering new AgentExecutor chain...
I need to find out who Marie Curie is and summarize her information in two sentences. I should use the `wikipedia_search` tool for this.
Action: wikipedia_search
Action Input: "Marie Curie"
Observation: Page: Marie Curie
Summary: Maria Salomea Skłodowska-Curie,​​ más conocida como Marie Curie​​ o Madame Curie (Varsovia, 7 de noviembre de 1867-Passy, 4 de julio de 1934), fue una física y química de origen polaco. Pionera en el campo de la radiactividad, es la primera y única persona en recibir dos premios Nobel en distintas especialidades científicas: Física y Química.​ También fue la primera mujer en ocupar el puesto de profesora en la Universidad de París y la primera en recibir sepultura con honores en el Panteón de París por méritos propios en 1995.​
Nació en Varsovia, en lo que entonces era el Zarato de Polonia (territorio administrado por el Imperio ruso). Estudió clandestinamente en la «universidad flotante» de Varsovia y comenzó su formaci